# UW-Madison GI Tract Image Segmentation

## Check model performance

In [29]:
#!/usr/bin/env python

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.options.plotting.backend = "plotly"
from tqdm.notebook import tqdm
tqdm.pandas()
import numpy as np
import os
import re
import cv2
from plotly.offline import init_notebook_mode
from glob import glob

import tensorflow as tf

# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image, ImageEnhance
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
from matplotlib.colors import LinearSegmentedColormap
import plotly
import PIL
import cv2

# Import Tensorflow and Keras
from tensorflow import keras
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

from keras.models import load_model


		– MATPLOTLIB VERSION: 3.5.2


In [30]:
%cd 'hpc_train_files'

[Errno 2] No such file or directory: 'hpc_train_files'
/Users/viktor/Documents/BA/GIT/HealthyOrganTracker/hpc_train_files


In [31]:
from utility import rle_encode, rle_decode, open_gray16
from config import CFG
from dataloader import DataGenerator
from loss import dice_coef, iou_coef, dice_loss, bce_dice_loss, dice_coef2
from datapreparation import extract_metadata, remove_faulties

## Config ⚙️


In [32]:
BASE_PATH = 'input/uw-madison-gi-tract-image-segmentation/'
TRAIN_DIR =  BASE_PATH +'train'
TRAIN_CSV =  BASE_PATH +'train.csv'
STYE = "multilabel"
DEBUG = True

"""
Backbones available: 
        'efficientnetb0'
        'efficientnetb1'
        'efficientnetb2'
        'efficientnetb3'
        'efficientnetb4'
        'efficientnetb5'
        'efficientnetb6'
        'efficientnetb7'
        'inceptionresnetv2'
        'inceptionv3'
        'resnet50'
        'resnext50'
        'resnext101'
        'seresnext50'
        'seresnext101'
        'densenet121'
        'densenet201'
        """

"\nBackbones available: \n        'efficientnetb0'\n        'efficientnetb1'\n        'efficientnetb2'\n        'efficientnetb3'\n        'efficientnetb4'\n        'efficientnetb5'\n        'efficientnetb6'\n        'efficientnetb7'\n        'inceptionresnetv2'\n        'inceptionv3'\n        'resnet50'\n        'resnext50'\n        'resnext101'\n        'seresnext50'\n        'seresnext101'\n        'densenet121'\n        'densenet201'\n        "

In [33]:
cfg = CFG(
    backbone            = 'efficientnetb0',
    img_dims            = (256,256,3),
    model               = 'U-Net',
    batch_size          = 16, 
    epochs              = 1, 
    kaggle              = False, 
    use_fold_csv        = True,
    semi3d_data         = False,
    remove_faulty_cases = True,
    use_crop_data       = False)

## Load prepared Dataframe 

In [34]:
DEBUG = False
if DEBUG:
    df_train["multilabel_mask_path"] = df_train.progress_apply(lambda _row: make_seg_mask(_row, outputdir, resize_to=SEG_SHAPE, resize=False), axis=1)
    df_train.to_csv("df_train_with_mask_paths.csv", index=False)
else:
    df_train = pd.read_csv("df_train_with_mask_paths.csv")
    df_train.fillna('', inplace=True)



In [35]:
df_train

,Unnamed: 0,id,large_bowel,small_bowel,stomach,path,case,day,slice,width,...,count,path00,path01,path02,image_paths,rs,re,cs,ce,multilabel_mask_path
0,0,case123_day20_slice_0001,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,1,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
1,1,case123_day20_slice_0002,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,2,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
2,2,case123_day20_slice_0003,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,3,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
3,3,case123_day20_slice_0004,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,4,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
4,4,case123_day20_slice_0005,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,5,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38491,38491,case30_day0_slice_0140,,,,input/uw-madison-gi-tract-image-segmentation/t...,30,0,140,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,1,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
38492,38492,case30_day0_slice_0141,,,,input/uw-madison-gi-tract-image-segmentation/t...,30,0,141,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,1,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
38493,38493,case30_day0_slice_0142,,,,input/uw-madison-gi-tract-image-segmentation/t...,30,0,142,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,1,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
38494,38494,case30_day0_slice_0143,,,,input/uw-madison-gi-tract-image-segmentation/t...,30,0,143,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,1,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...


In [36]:
for file in glob(os.path.join("../tensorboard_logs/efb2_trains/batch_size_16", "**", "historyFold3.csv"), recursive=True):
    if file.endswith(".csv"):
        print(file)

../tensorboard_logs/efb2_trains/batch_size_16/UNet_BB_efficientnetb7_DIM_(256, 256, 3)_SEMI3D_False_CROPDATA_False_FAULTIES_True_BATCH_16_EPOCHS_30_FOLD_3.h5_17082022-1833_FOLD_3/historyFold3.csv
../tensorboard_logs/efb2_trains/batch_size_16/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_True_BATCH_16_EPOCHS_50_FOLD_3.h5_17082022-2314_FOLD_3/historyFold3.csv
../tensorboard_logs/efb2_trains/batch_size_16/UNet_BB_efficientnetb7_DIM_(256, 256, 3)_SEMI3D_False_CROPDATA_True_FAULTIES_True_BATCH_16_EPOCHS_30_FOLD_3.h5_17082022-1833_FOLD_3/historyFold3.csv
../tensorboard_logs/efb2_trains/batch_size_16/UNet_BB_efficientnetb7_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_True_BATCH_16_EPOCHS_30_FOLD_3.h5_17082022-1833_FOLD_3/historyFold3.csv
../tensorboard_logs/efb2_trains/batch_size_16/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_False_CROPDATA_False_FAULTIES_True_BATCH_16_EPOCHS_30_FOLD_3.h5_17082022-1837_FOLD_3/historyFold3.csv
../tensorboard_logs/efb2_

In [37]:
def get_performance_from_history_df(path="tensorboard_logs/", output="all_histories.csv", extra_regex=False):
    if not extra_regex:
        all_histories = pd.DataFrame(columns=['Encoder','Größe' ,'Beste Epoche', 'Loss', 'Val. loss', 'Dice', 'Val. Dice', 'IoU', 'Val. IoU'])
    else:
        all_histories = pd.DataFrame(columns=['Encoder','Größe' ,'Beste Epoche', 'Loss', 'Val. loss', 'Dice', 'Val. Dice', 'IoU', 'Val. IoU', '2.5D', 'Crop' , 'Bereinigt', 'Batch Size', 'Epochen'])

    # Iterate over every historyFold3.csv in tensorboard_logs/base_BS_16_no_add_ons/subfolder and plot each column values with plotly and mark the maximum value and drop the first column
    for file in glob(os.path.join(path, "**", "historyFold3.csv"), recursive=True):
        if file.endswith(".csv"):
            print(file)
            history_df = pd.read_csv(file, index_col=0)
            
            regexBone, regexDim, threeDim, regexCrop, regexBatch, regexEpochs, regexFold, regexFaulties = get_meta_info_from_model_path(file)
            
            # print all regex parameters 
            print(regexBone, regexDim, threeDim, regexCrop, regexBatch, regexEpochs, regexFold, regexFaulties)

            best_epoch = get_best_epoch(history_df)

            loss = history_df.loc[best_epoch, 'loss']
            val_loss = history_df.loc[best_epoch, 'val_loss']

            dice_coef = history_df.loc[best_epoch, 'dice_coef']
            val_dice_coef = history_df.loc[best_epoch, 'val_dice_coef']

            iou_coef = history_df.loc[best_epoch, 'iou_coef']
            val_iou_coef = history_df.loc[best_epoch, 'val_iou_coef']

            if not extra_regex:
                all_histories = all_histories.append({'Encoder':regexBone, 'Größe':regexDim, 'Beste Epoche':best_epoch, 'Loss':loss, 'Val. loss':val_loss, 'Dice':dice_coef, 'Val. Dice':val_dice_coef, 'IoU':iou_coef, 'Val. IoU':val_iou_coef}, ignore_index=True)
            else:
                all_histories = all_histories.append({'Encoder':regexBone, 'Größe':regexDim, 'Beste Epoche':best_epoch, 'Loss':loss, 'Val. loss':val_loss, 'Dice':dice_coef, 'Val. Dice':val_dice_coef, 'IoU':iou_coef, 'Val. IoU':val_iou_coef, '2.5D':threeDim, 'Crop':regexCrop,'Bereinigt':regexFaulties ,'Batch Size':regexBatch, 'Epochen':regexEpochs}, ignore_index=True)

    all_histories = all_histories.sort_values(by=['Val. Dice'], ascending=False)
    all_histories.to_csv(output)

    return all_histories


def get_best_epoch(history_df):
    return history_df[history_df["val_loss"]==history_df["val_loss"].min()].index.values[0]

def print_best_epoch(history_df):
    best_epoch = get_best_epoch(history_df)
    print(f"Best epoch: {best_epoch}")
    print(f"Best loss: {history_df.loc[best_epoch, 'val_loss']}")

def print_history_latex(history_df):

    print(history_df.to_latex(index=False , float_format=lambda x: '{:.4f}'.format(x)))

def get_meta_info_from_model_path(file):
    regexBone = 'None' if not re.search("BB_(\w+)_D",file) else re.search("BB_(\w+)_D",file).group(1) 
    regexDim = 'None' if not re.search("DIM_\((\d+),",file) else int(re.search("DIM_\((\d+),",file).group(1) )
    threeDim = 'None' if not re.search("SEMI3D_([aA-zZ]{4,5})_",file) else re.search("SEMI3D_([aA-zZ]{4,5})_",file).group(1) 
    regexCrop = 'None' if not re.search("CROPDATA_([aA-zZ]{4,5})_",file) else re.search("CROPDATA_([aA-zZ]{4,5})_",file).group(1) 
    regexBatch = 'None' if not re.search("BATCH_(\d+)_",file) else int(re.search("BATCH_(\d+)_",file).group(1) )
    regexEpochs = 'None' if not re.search("EPOCHS_(\d+)_",file) else int(re.search("EPOCHS_(\d+)_",file).group(1))
    regexFold = 'None' if not re.search("Fold_(\d).",file) else int(re.search("Fold_(\d).",file).group(1))
    regexFaulties = 'None' if not re.search("FAULTIES_([aA-zZ]{4,5})_",file) else re.search("FAULTIES_([aA-zZ]{4,5})_",file).group(1)

    return regexBone, regexDim, threeDim, regexCrop, regexBatch, regexEpochs, regexFold, regexFaulties


In [38]:
histories_bs16 =  get_performance_from_history_df(path="../tensorboard_logs/efb2_trains/batch_size_16", output="all_histories.csv", extra_regex=True)
histories_bs32 =  get_performance_from_history_df(path="../tensorboard_logs/efb2_trains/batch_size_32", output="all_histories.csv", extra_regex=True)

../tensorboard_logs/efb2_trains/batch_size_16/UNet_BB_efficientnetb7_DIM_(256, 256, 3)_SEMI3D_False_CROPDATA_False_FAULTIES_True_BATCH_16_EPOCHS_30_FOLD_3.h5_17082022-1833_FOLD_3/historyFold3.csv
efficientnetb7 256 False False 16 30 None True
../tensorboard_logs/efb2_trains/batch_size_16/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_True_BATCH_16_EPOCHS_50_FOLD_3.h5_17082022-2314_FOLD_3/historyFold3.csv
efficientnetb2 256 True True 16 50 None True
../tensorboard_logs/efb2_trains/batch_size_16/UNet_BB_efficientnetb7_DIM_(256, 256, 3)_SEMI3D_False_CROPDATA_True_FAULTIES_True_BATCH_16_EPOCHS_30_FOLD_3.h5_17082022-1833_FOLD_3/historyFold3.csv
efficientnetb7 256 False True 16 30 None True
../tensorboard_logs/efb2_trains/batch_size_16/UNet_BB_efficientnetb7_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_True_BATCH_16_EPOCHS_30_FOLD_3.h5_17082022-1833_FOLD_3/historyFold3.csv
efficientnetb7 256 True True 16 30 None True
../tensorboard_logs/efb2_trains/batch_

In [43]:
histories_bs16[histories_bs16['Encoder'] == 'efficientnetb7']

,Encoder,Größe,Beste Epoche,Loss,Val. loss,Dice,Val. Dice,IoU,Val. IoU,2.5D,Crop,Bereinigt,Batch Size,Epochen
0,efficientnetb7,256,23,0.036749,0.088244,0.91483,0.803611,0.916514,0.841307,False,False,True,16,30
3,efficientnetb7,256,20,0.038981,0.111671,0.910055,0.74255,0.882917,0.833372,True,True,True,16,30
7,efficientnetb7,256,29,0.035209,0.117446,0.91828,0.730084,0.911749,0.83255,True,False,True,16,30
2,efficientnetb7,256,22,0.038756,0.118247,0.910607,0.727932,0.896532,0.828215,False,True,True,16,30


In [44]:
print_history_latex(histories_bs16)

\begin{tabular}{llllllllllllll}
\toprule
       Encoder & Größe & Beste Epoche &   Loss & Val. loss &   Dice & Val. Dice &    IoU & Val. IoU &  2.5D &  Crop & Bereinigt & Batch Size & Epochen \\
\midrule
efficientnetb7 &   256 &           23 & 0.0367 &    0.0882 & 0.9148 &    0.8036 & 0.9165 &   0.8413 & False & False &      True &         16 &      30 \\
efficientnetb2 &   256 &           26 & 0.0359 &    0.0907 & 0.9171 &    0.7979 & 0.8851 &   0.8418 &  True &  True &      True &         16 &      50 \\
efficientnetb2 &   256 &           28 & 0.0347 &    0.1033 & 0.9200 &    0.7678 & 0.9182 &   0.8255 & False &  True &      True &         16 &      30 \\
efficientnetb2 &   256 &           26 & 0.0353 &    0.1085 & 0.9182 &    0.7563 & 0.9222 &   0.8242 & False & False &      True &         16 &      30 \\
efficientnetb7 &   256 &           20 & 0.0390 &    0.1117 & 0.9101 &    0.7425 & 0.8829 &   0.8334 &  True &  True &      True &         16 &      30 \\
efficientnetb2 &   256 &  